# Rasterio: Mt. Rainier glaciers example

This exercise focuses on working with [GIS raster data](https://docs.qgis.org/3.4/en/docs/gentle_gis_introduction/raster_data.html). It is based on a [GeoHackWeek tutorial](https://github.com/geohackweek/tutorial_contents/blob/master/raster/notebooks/rainier_dem_example.ipynb). We will use the `rasterio` and `rasterstats` libraries, along with `numpy` and `matplotlib` to look changes in the surface elevations of mapped glaciers since 1970.

#### Key takeaways:
* raster formats such as GeoTiff map array data on a regular grid using a simple transform (offset, rotation and pixel size).
* working with raster data in python is inherently a little difficult, because the raster data model is designed to work efficiently with large datasets on disk (without loading whole thing into memory). Other available tools, such as the [gdal commandline tools](https://gdal.org/programs/index.html) or ArcPy macros, tend to stay in this paradigm by taking files as input to fuctions that write files as output.
* conversely, when we work with arrays in-memory with `numpy`, we typically think it terms of rows and columns

#### The main idea is to   
   1) use `rasterio` (or another tool) to handle the files and any transformations needed to align the data. As we'll see below, most of these operations aren't especially intuitive or easy to remember. Thankfully, there are recipes available online that are easy to copy for performing common tasks. The [rasterio manual](https://rasterio.readthedocs.io/en/stable/) is a great place to start.  
   2) once the data are in numpy, use `numpy` and other tools in the scientific python stack (e.g. `scipy`) to do the computations  

#### Datasets:
* 3 Digital Elevation Models (DEMs) of Mt. Rainier elevations in 1970, 2008 and 2015; each on a different grid.
* shapefile of glacier extents

#### Operations:
* read and write rasters to/from `numpy` arrays
* plotting raster data
* warping (resampling) raster data to different grids
* reprojecting raster data to different coordinate reference systems
* getting raster data at discrete points
* zonal statistics
* mapping vector features onto rasters ("rasterizing")

#### References:
* The [rasterio manual](https://rasterio.readthedocs.io/en/stable/)
* The [GeoHackWeek tutorials](https://geohackweek.github.io/raster/) for more information on working with raster data in the python

In [ ]:
import os
import numpy as np
import geopandas as gp
import rasterio
from rasterio.plot import show
from rasterio import features
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource

# for elevation hillshade if we want to use it
ls = LightSource(azdeg=315, altdeg=45)

### Define out input data

We'll be working with three rasters and a single shapefile for this exercise

In [ ]:
data_path = os.path.join("data", "rasterio")

dem_file_1970 = os.path.join(data_path, "19700901_ned1_2003_adj_warp.tif")
dem_file_2008 = os.path.join(data_path, "20080901_rainierlidar_30m-adj.tif")
dem_file_2015 = os.path.join(data_path, "20150818_rainier_summer-tile-30.tif")

glaciers_shapefile = os.path.join(data_path, "rgi60_glacierpoly_rainier.shp")

input_rasters = {
    1970: dem_file_1970,
    2008: dem_file_2008,
    2015: dem_file_2015
}

### Quickly view a raster with rasterio

In [ ]:
with rasterio.open(input_rasters[1970]) as src:
    show(src)

### Get raster values at point locations

if we have a list of (x, y) tuples, we can unpack it into two lists of x and y values:
`zip(*points)`  

`zip(*points)` returns a tuple of the two lists

First let's define visualize our point locations on our raster

In [ ]:
points = [
    (594508.70, 5189550.08), # summit, in utm zone 10N
    (596702, 5187675) # camp muir
]
x, y = zip(*points)
fig, ax = plt.subplots(figsize=(8, 8))
with rasterio.open(input_rasters[2015]) as src:
     ax = show(src, ax=ax, zorder=1, cmap="plasma")

ax.scatter(x, y, c="k", s=50, zorder=5)
ax.set_facecolor("lightgrey")
plt.colorbar(ax.images[0], shrink=0.7);

#### Now let's sample elevations at these points

If we do the unpacking in a function call (e.g. `src.index(*zip(*points))`), we need to add a leading `*` to unpack the tuple into two separate arguments

In [ ]:
with rasterio.open(input_rasters[2015]) as src:       
        meta = src.meta
        data = src.read(1)
        i, j = src.index(*zip(*points))
print(data[i, j])

### Viewing the 3 rasters side by side as subplots

The data are all on different grids with varying extents and projections. Let's plot them up to visualize this.


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(21, 7))

meta = {}
for i, (year, f) in enumerate(input_rasters.items()):
    with rasterio.open(f) as src:
        
        # get the metadata
        meta[year] = src.meta
        
        # plot the data
        if i == 0:
            data = src.read(1)
            vmin, vmax = data.min(), data.max()
        ax=axes.flat[i]
        show(src, ax=ax, vmin=vmin, vmax=vmax)
        ax.set_title(year)
        ax.set_facecolor("lightgrey")
        plt.colorbar(ax.images[0], ax=ax, 
                     orientation='horizontal', label='elevation, m')

We see what these are by fetching the metadata dictionary (`src.meta` attribute) for each one. Each grid is described in the metadata:

In [ ]:
meta

### Warp the 1970 and 2008 rasters onto the 2015 grid

The code below was copied from the [rasterio manual](https://rasterio.readthedocs.io/en/stable/topics/virtual-warping.html). The result is a new set of files aligned to the destination grid.

In [ ]:
from rasterio.enums import Resampling
from rasterio.vrt import WarpedVRT
from rasterio import shutil as rio_shutil


vrt_options = {
    'resampling': Resampling.cubic,
    'crs': meta[2015]['crs'],
    'transform': meta[2015]['transform'],
    'height': meta[2015]['height'],
    'width': meta[2015]['width'],
}

aligned_rasters = {}
for year, path in input_rasters.items():

    with rasterio.open(path) as src:

        with WarpedVRT(src, **vrt_options) as vrt:

            # At this point 'vrt' is a full dataset with dimensions,
            # CRS, and spatial extent matching 'vrt_options'.

            # Read all data into memory.
            data = vrt.read()

            # Process the dataset in chunks.  Likely not very efficient.
            for _, window in vrt.block_windows():
                data = vrt.read(window=window)

            # Dump the aligned data into a new file.  A VRT representing
            # this transformation can also be produced by switching
            # to the VRT driver.
            data_path, name = os.path.split(path)
            outfile = os.path.join(data_path, f'aligned-{name}')
            rio_shutil.copy(vrt, outfile, driver='GTiff')
            print('wrote {}'.format(outfile))
            aligned_rasters[year] = outfile

#### Now let's make sure all three are on the same grid

In [ ]:
meta = {}
for year, f in aligned_rasters.items():
    with rasterio.open(f) as src:
        meta[year] = src.meta
meta

Success!

### Read the aligned grids into a new set of masked numpy arrays

`rasterio` can be used to construct masked numpy arrays based on the `'nodata'` value in the raster metadata

In [ ]:
aligned_data = {}
for year, f in aligned_rasters.items():
    with rasterio.open(f) as src:
        data = src.read(1, masked=True)
        aligned_data[year] = data

In [ ]:
aligned_data[2015]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
im = ax.imshow(aligned_data[2015])
ax.set_facecolor("lightgrey")
plt.colorbar(im, shrink=0.7);

### Assign information on shared raster grid to variables for later use

In [ ]:
epsg_32610 = meta[2015]['crs']
transform = meta[2015]['transform']
nrow, ncol = aligned_data[2015].shape
cellsize = transform[0]
extent = src.bounds[0], src.bounds[2], src.bounds[1], src.bounds[3]  # extent for plotting (xmin, xmax, ymin, ymax)

### Get the summit elevation

Ss noted in the original GeoHackWeek example, the elevation values in these rasters are relative to the WGS84 datum,
not a geoid model that approximate sea level. The geoid offset at Rainier is approximately 18.79 m, meaning that we have to add we have to add this to the values to get the [correct summit elevation of 14,411 ft](https://en.wikipedia.org/wiki/Mount_Rainier).

In [ ]:
summit_ij = np.unravel_index(aligned_data[1970].argmax(), aligned_data[1970].shape)
summit_ij

The corrected summit elevations being a little under 14,411 is a little unsatisfying, but one of the costs of downsampling the DEMs to a 30 m resolution to conserve space. At any rate, since we are interested in differences, the geoid offset doesn't matter for the remainder of our analysis.

In [ ]:
elevation_offset = 18.79
(aligned_data[1970][summit_ij] + elevation_offset) * 3.2084  # in feet

### Compute the elevation differences

In [ ]:
elevation_diffs = {}
elevation_diffs['1970-2008'] = aligned_data[2008] - aligned_data[1970]
elevation_diffs['2008-2015'] = aligned_data[2015] - aligned_data[2008]
elevation_diffs['1970-2015'] = aligned_data[2015] - aligned_data[1970]

fig, axes = plt.subplots(1, 3, figsize=(21, 7))

for i, (dates, diffs) in enumerate(elevation_diffs.items()):

    ax=axes.flat[i]
    im = ax.imshow(diffs, cmap='RdBu', vmin=-30, vmax=30)
    ax.set_facecolor("lightgrey")
    ax.set_title(dates)

fig.colorbar(im, ax=axes.flat, label='elevation change, m', shrink=0.6, extend='both');

### Average rates of change

In [ ]:
elevation_rates = {}
for dates, diffs in elevation_diffs.items():
    yr0, yr1 = map(int, dates.split('-'))
    annual_rate = diffs/(yr1-yr0)
    elevation_rates[dates] = annual_rate
    
fig, axes = plt.subplots(1, 3, figsize=(21, 7))

for i, (dates, rates) in enumerate(elevation_rates.items()):

    ax=axes.flat[i]
    im = ax.imshow(rates, cmap='RdBu', vmin=-2, vmax=2)
    ax.set_facecolor("lightgrey")
    ax.set_title(dates)

fig.colorbar(im, ax=axes.flat, label='average rate of change, m/yr', shrink=0.6, extend='both');

Notice the huge changes around the margins of the mountain. These are due to the 1970 and 2008 DEMs representing "bare ground", while the 2015 DEM includes trees.

### Map the shapefile of glacier extents onto the same grid as the rasters
* first reproject the glacier features to the same crs as the rasters
* then make a column of unique identifiers (integers) for relating the features to raster pixels

In [ ]:
glaciers = gp.read_file(glaciers_shapefile)
glaciers.head()

In [ ]:
glaciers.to_crs(epsg_32610, inplace=True)
glaciers['id'] = np.arange(1, len(glaciers) + 1)

### Use rasterio to rasterize the glacier extents
* input is a collection of the (feature geometry, unique value) tuples and an [Affine transform](https://github.com/rasterio/affine). Futher reading on affine transforms can be found [here](https://en.wikipedia.org/wiki/Affine_transformation).
* output is a numpy array 

In [ ]:
geoms = zip(glaciers.geometry, glaciers.id)
glacier_footprints = features.rasterize(geoms, out_shape=(nrow, ncol), transform=transform)

In [ ]:
mask = glacier_footprints == 0
masked_footprint = np.ma.masked_array(glacier_footprints, mask=mask)

# plot footprints
fig, ax = plt.subplots(figsize=(8, 8))
im = ax.imshow(masked_footprint)
ax.set_facecolor("lightgrey")
plt.colorbar(im, shrink=0.7);

### Clip the 1970-2015 raster to get an estimate of total volume change for all glaciers

Using the mask we created before, we can clip the elevation difference arrays to estimate the total volume of change for all of the glaciers

In [ ]:
masked_diffs = np.ma.masked_array(elevation_diffs["1970-2015"], mask=mask)

In [ ]:
total_vol_change = masked_diffs.sum() * cellsize ** 2 / 1000**3

fig, ax = plt.subplots(figsize=(8, 8))
im = plt.imshow(masked_diffs, cmap='RdBu', vmin=-80, vmax=80)

ax.text(10, 50, "Total volume change: {:.2f} km$^3$".format(total_vol_change))
ax.set_facecolor("lightgrey")
plt.colorbar(im, label='change in m', shrink=0.6, extend='both');

### Get volume change by glacier, see if it relates to minimum elevation

In [ ]:
vol_change = []
for n in glaciers.id.values:
    # compute total volume change
    change_1970_2015 = elevation_diffs['1970-2015'][glacier_footprints == n]
    vc = change_1970_2015.sum() * cellsize ** 2 / 1000**3
    vol_change.append(vc)
    
glaciers['d_vol_km3'] = vol_change

In [ ]:
glaciers.sort_values(by='d_vol_km3', inplace=True)
glaciers.head()

#### Now we can plot the volume of change vs minimum elevation `'Zmin'`

In [ ]:
fig, ax = plt.subplots(figsize=(11, 8.5))
s = ax.scatter(glaciers.Zmin, glaciers.d_vol_km3, c=glaciers.d_vol_km3, cmap='plasma_r')
for i, r in glaciers.head(10).iterrows():
    ax.text(r.Zmin * 1.03, r.d_vol_km3, r.Name)
ax.set_ylabel('Volume change, 1970-2015, km$^3$')
ax.set_xlabel('Minimum elevation, in meters')
plt.colorbar(s, label='change in km$^3$');

We can also plot this relationship spatially using geopandas

In [ ]:
fig, ax = plt.subplots(figsize=(11, 8.5))

ax.imshow(ls.hillshade(aligned_data[1970], vert_exag=0.5), cmap='gray', extent=extent)
ax = glaciers.plot('d_vol_km3', ax=ax, legend=True, zorder=10, alpha=0.5, cmap="plasma_r")

If we regress by area, we see that volume change is coorelated to the original area of the glacier

In [ ]:
fig, ax = plt.subplots(figsize=(11, 8.5))
s = ax.scatter(glaciers.Area, glaciers.d_vol_km3, c=glaciers.d_vol_km3, cmap='plasma_r')
for i, r in glaciers.head(10).iterrows():
    ax.text(r.Area - 2.03, r.d_vol_km3 - 0.003, r.Name)
ax.set_ylabel('Volume change, 1970-2015, km$^3$')
ax.set_xlabel('Glacial area in km$^2$')
plt.colorbar(s, label='change in km$^3$');

Lets plot only the elevation differences in glaciers now

In [ ]:
just_glaciers = np.ma.masked_array(elevation_diffs['1970-2015'], mask=mask)
fig, ax = plt.subplots(figsize=(11, 8.5))

ax.imshow(
    ls.hillshade(
        aligned_data[1970], 
        vert_exag=0.2
    ), 
    cmap='gray', 
    zorder=-1
)
plt.imshow(just_glaciers, cmap='RdBu', vmin=-30, vmax=30, alpha=0.8)
plt.colorbar(label='Change in glacier elevation, 1970-2015, meters')

### Writing a raster: save out 1970-2015 elevation changes

To do this, we need to provide rasterio with the same information we got when we read the metadata for the aligned rasters (the elevation changes are on the same grid).
* first reset the mask to include all areas outside of the glacier footprints
* to simplify things, we can just copy the metadata dict we already have, and feed it to `rasterio.open` as keyword arguments by unpacking it (`**` notation)
    * let's add compression though to reduce the file size
* we can have rasterio write the mask as well
* Note that a raster can have more than one band. [This code (see `export_array` function)](https://github.com/aleaf/modflow-export/blob/master/mfexport/array_export.py) illustrates writing a 3D raster with a 3D mask to a single GeoTiff with multiple bands.

In [ ]:
elevation_diffs['1970-2015'] = np.ma.masked_array(elevation_diffs['1970-2015'], mask=mask)

In [ ]:
out_meta = meta[2015].copy()
out_meta['compress'] = 'lzw'
out_meta

In [ ]:
el_change_outfile = os.path.join(data_path, 'rainier_glaciers_1970-2015_el_change_m.tif')
with rasterio.open(el_change_outfile, 'w', **out_meta) as dest:
    
    # write the data to band 1
    dest.write(elevation_diffs['1970-2015'], 1)
    
    # write the mask as well
    dest.write_mask(~elevation_diffs['1970-2015'].mask)

### What if we want to reproject to a different CRS?

* We already have the information we need in memory, but it easiest to simply open a raster with the source grid and then read the metadata from the rasterio file instance
* We'll then feed that information to the `rasterio.warp.calculate_default_transform` function, which will return a new orientation (transform instance) and dimensions for the raster in the destination CRS.
* We'll use the new location information to update the metadata from the source grid
* The code below was taken from the [rasterio manual](https://rasterio.readthedocs.io/en/stable/topics/reproject.html) section on reprojection

In [ ]:
out_meta

In [ ]:
from rasterio.crs import CRS
from rasterio.warp import calculate_default_transform, reproject, Resampling

dst_crs = 'epsg:4269'
with rasterio.open(el_change_outfile) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)
    
    # copy the source metadata
    kwargs = src.meta.copy()
    
    # update it with the new location information
    kwargs.update({
    'crs': dst_crs,
    'transform': transform,
    'width': width,
    'height': height
})
outfile = aligned_rasters[1970].replace('warp.tif', '4269.tif')
outfile

In [ ]:
with rasterio.open(outfile, 'w', **kwargs) as dst:
    reproject(
        source=aligned_data[1970],
        destination=rasterio.band(dst, 1),
        src_transform=src.transform,
        src_crs=src.crs,
        dst_transform=transform,
        dst_crs=dst_crs,
        resampling=Resampling.bilinear)

## Zonal statistics with the `rasterstats` package

(another way to reduce raster data by polygon features, as we did above in summing the elevation differences by glacier)

In [ ]:
from rasterstats import zonal_stats

In [ ]:
help(zonal_stats)

In [ ]:
from rasterstats import gen_zonal_stats
help(gen_zonal_stats)

In [ ]:
result = zonal_stats(glaciers.geometry.tolist(), el_change_outfile, stats=['mean'])

In [ ]:
result[0:10]

In [ ]:
glaciers["mean_el_change_m"] = [d["mean"] for d in result]

In [ ]:
glaciers.sort_values(by="mean_el_change_m").head()